In [1]:
import pystac_client
import planetary_computer as pc
import geopandas as gpd
from shapely.geometry import Point

In [2]:
# some helper functions
def read_window(reader, bounds):
    bounds_ = warp.transform_bounds("epsg:4326", reader.crs, *bounds.bounds)
    window = windows.from_bounds(*bounds_, transform=reader.transform)
    return window, reader.read(window=window)

def update_profile(reader, bounds):      
    profile = reader.profile
    profile.update({
        "height": window.height,
        "width": window.width,
        "transform": windows.transform(window, reader.transform)
    })
    return profile

In [3]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=pc.sign_inplace,
)

bounds = Point(86.55, 28).buffer(0.05)
date_range = "2021-01-01/2021-12-31"

In [4]:
constraints = {"eo:cloud_cover": {"lt": 20}, "s2:nodata_pixel_percentage": {"lt": 10}}
search_results = catalog.search(
    collections="sentinel-2-l2a", 
    intersects=bounds, 
    datetime=date_range, 
    query=constraints
)

In [5]:
import rasterio
from rasterio import windows
from rasterio import warp

items = search_results.item_collection()
link = items[0].assets["visual"].href # would be worth downloading the other assets too

## reading and writing the data
with rasterio.open(link) as reader:
    window, band_data = read_window(reader, bounds)
    profile = update_profile(reader, window)
    with rasterio.open("output.tiff", "w", **profile) as writer:
        writer.write(band_data)

In [6]:
!gdalwarp -s_srs epsg:32645 -t_srs epsg:4326 output.tiff output-warped.tiff

Creating output file that is 1055P x 1053L.
Processing output.tiff [1/1] : 0Using internal nodata values (e.g. 0) for image output.tiff.
Copying nodata values from source output.tiff to destination output-warped.tiff.
...10...20...30...40...50...60...70...80...90...100 - done.
